#**Capital bike diaries**

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install pygeohash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00


In [3]:
import os
from pathlib import Path
import scipy
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import sklearn
import folium
import pygeohash as pgh

##Cleaning and data integration

In [36]:
!cp -r "/content/drive/MyDrive/Dataset data" /content/


In [19]:
folder_path = Path('/content/Dataset data/')
#بيانات الرحلات اليومية
df_rent = pd.read_parquet(folder_path / "daily-rent.parquet") #df_rent: يحتوي على كافة الرحلات اليومية لاستئجار الدراجات
df_sampled_rent = pd.read_parquet(folder_path / "daily-rent-sampled.parquet") # نسخة مُصغّرة أو عيّنة من df_rent
#بيانات الطقس
df_weather = pd.read_csv(folder_path / "Washington,DC,USA 2024-01-01 to 2024-12-31.csv") # يحتوي على بيانات الطقس اليومية في واشنطن العاصمة من 1 يناير إلى 31 ديسمبر 2024.
#بيانات المحطات
df_stations_table = pd.read_csv(folder_path / "stations.csv") #جدول إضافي للمحطات، قد يحتوي على تفاصيل أخرى مثل معرّف المحطة (ID)، الاسم، عدد الأرصفة، إلخ
df_stations_info = pd.read_csv(folder_path / "Capital_Bikeshare_Locations.csv")  #معلومات حول مواقع محطات استئجار الدراجات الإحداثيات، اسم المحطة، إلخ.
#بيانات جغرافية ومناطق
df_cbd = gpd.read_file(folder_path / "DDOT_Central_Business_District.geojson") # المنطقة التجارية المركزية – تُستخدم لتحديد ما إذا كانت الرحلة بدأت أو انتهت داخل هذه المنطقة.
df_zones = gpd.read_file(folder_path / "Residential_and_Visitor_Parking_Zones.geojson") #مناطق وقوف السيارات السكنية والزوار (Residential and Visitor Parking Zones).
#بيانات النقل العام
df_metro = pd.read_csv(folder_path / "Metro_Bus_Stops.csv") #مواقف حافلات المترو (Metro Bus Stops).
df_shuttle = pd.read_csv(folder_path / "Shuttle_Bus_Stops.csv") # مواقف حافلات النقل الصغيرة (Shuttle Bus Stops).



In [5]:
df_rent.shape

(6114323, 13)

In [6]:
df_sampled_rent.shape

(2018456, 13)

In [7]:
df_rent.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [8]:
df_rent.dtypes

,0
ride_id,string[python]
rideable_type,string[python]
started_at,datetime64[ns]
ended_at,datetime64[ns]
start_station_name,string[python]
start_station_id,string[python]
end_station_name,string[python]
end_station_id,string[python]
start_lat,float64
start_lng,float64


In [9]:
df_rent.nunique()

,0
ride_id,6114182
rideable_type,2
started_at,5866403
ended_at,5869571
start_station_name,818
start_station_id,803
end_station_name,818
end_station_id,803
start_lat,468450
start_lng,501510


In [41]:
df_rent.tail(40)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
6114283,D04E61A89DC59793,electric_bike,2024-12-14 15:11:39.759,2024-12-14 15:25:21.882,2nd St & Massachusetts Ave NE,31641,14th & Belmont St NW,31119,38.894972,-77.003135,38.921074,-77.031887,casual
6114284,1438F014784DB8CE,electric_bike,2024-12-14 16:16:36.260,2024-12-14 16:56:29.945,Columbia Pike & W&OD Trail,31981,Columbia Pike & W&OD Trail,31981,38.856645,-77.109609,38.860000,-77.110000,member
6114285,C61A01D2C820CC49,electric_bike,2024-12-20 20:29:56.705,2024-12-20 20:36:56.679,Eisenhower Ave & Mill Race Ln,31082,Ballenger Ave & Dulaney St,31084,38.801111,-77.068952,38.800000,-77.070000,casual
6114286,F0C83E34822CF97A,electric_bike,2024-12-07 16:33:04.195,2024-12-07 16:44:13.034,Georgia Ave & Dahlia St NW,31425,Takoma Metro,31408,38.976061,-77.026747,38.970000,-77.000000,casual
6114287,30CBD50DAFE5C2F3,electric_bike,2024-12-20 15:01:19.033,2024-12-20 16:22:30.497,Eisenhower Ave & Mill Race Ln,31082,Ballenger Ave & Dulaney St,31084,38.801111,-77.068952,38.780000,-77.090000,casual
6114288,68A387F43C4115A5,electric_bike,2024-12-13 13:53:14.924,2024-12-13 14:00:13.731,Georgia Ave & Dahlia St NW,31425,Takoma Metro,31408,38.976061,-77.026747,38.970000,-77.030000,casual
6114289,CF3CE77EC5527B12,electric_bike,2024-12-19 08:30:47.862,2024-12-19 08:33:09.694,Georgia & Missouri Ave NW,31411,Georgia & Missouri Ave NW,31411,38.961339,-77.027855,38.960000,-77.020000,member
6114290,1B8E18E872E92BD1,electric_bike,2024-12-10 16:03:21.399,2024-12-10 16:10:12.046,Georgia Ave & Dahlia St NW,31425,Takoma Metro,31408,38.976061,-77.026747,38.990000,-77.030000,member
6114291,3430289CD7D539B3,electric_bike,2024-12-15 18:31:05.881,2024-12-15 18:36:10.246,7th & K St NE,31660,M St & Delaware Ave NE,31627,38.902594,-76.995141,38.900000,-76.980000,member
6114292,520F226CAE9B3165,electric_bike,2024-12-17 08:31:26.395,2024-12-17 08:34:15.611,Georgia & Missouri Ave NW,31411,Georgia & Missouri Ave NW,31411,38.961339,-77.027855,38.960000,-77.020000,member


In [11]:
print(df_rent.duplicated().sum())

0


In [42]:
df_rent.isnull().sum()

,0
ride_id,0
rideable_type,0
started_at,0
ended_at,0
start_station_name,0
start_station_id,0
end_station_name,0
end_station_id,0
start_lat,0
start_lng,0


In [13]:
print((df_rent['start_station_id'] == df_rent['end_station_id']).mean())

0.0441441028702274


In [37]:
multi_names = (
    df_rent.dropna(subset=['end_station_id', 'end_station_name'])
    .groupby('end_station_id')['end_station_name']
    .nunique()
)

print(multi_names[multi_names > 1])


end_station_id
31045    2
31133    3
31225    2
31339    2
31380    2
31401    2
31407    2
31502    2
31528    2
31613    2
31660    2
31699    2
31726    2
31949    2
32288    2
Name: end_station_name, dtype: int64


In [36]:
only_name_missing = df_rent['start_station_name'].isna() & df_rent['start_station_id'].notna()
only_id_missing   = df_rent['start_station_id'].isna() & df_rent['start_station_name'].notna()
both_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna()
none_missing      = df_rent['start_station_id'].notna() & df_rent['start_station_name'].notna()
both4_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].isna() & df_rent['end_station_name'].isna()

both23_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].notna() & df_rent['end_station_name'].isna()
both32_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].isna() & df_rent['end_station_name'].notna()
both2_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].notna() & df_rent['end_station_name'].notna()
eonly_name_missing = df_rent['end_station_name'].isna() & df_rent['end_station_id'].notna()
eonly_id_missing   = df_rent['end_station_id'].isna() & df_rent['end_station_name'].notna()
eboth_missing  = df_rent['end_station_id'].isna() & df_rent['end_station_name'].isna()
print("✅ عدد الصفوف التي فقط الاسم مفقود:", only_name_missing.sum())
print("✅ عدد الصفوف التي فقط المعرف مفقود:", only_id_missing.sum())
print("✅ عدد الصفوف التي كلاهما مفقود:", both_missing.sum())
print("✅ عدد الصفوف التي لا يوجد بها فقد:", none_missing.sum())

print("✅td  عدد الصفوف التي  يوجد بها فقد:", both4_missing.sum())

print("✅td  عدد الصفوف التي  يوجد بها فقد:", both23_missing.sum())
print("✅td  عدد الصفوف التي  يوجد بها فقد:", both32_missing.sum())
print("✅td  عدد الصفوف التي  يوجد بها فقد:", both2_missing.sum())
print("✅ عدد الصفوف التي فقط الاسم مفقود:", eonly_name_missing.sum())
print("✅ عدد الصفوف التي فقط المعرف مفقود:", eonly_id_missing.sum())
print("✅ عدد الصفوف التي كلاهما مفقود:", eboth_missing.sum())

✅ عدد الصفوف التي فقط الاسم مفقود: 0
✅ عدد الصفوف التي فقط المعرف مفقود: 0
✅ عدد الصفوف التي كلاهما مفقود: 0
✅ عدد الصفوف التي لا يوجد بها فقد: 6114323
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅ عدد الصفوف التي فقط الاسم مفقود: 0
✅ عدد الصفوف التي فقط المعرف مفقود: 0
✅ عدد الصفوف التي كلاهما مفقود: 0


In [25]:
# تعبئة صفوف محطة البداية فقط عند فقدان كلي (اسم ومعرّف)
most_common_start = df_rent[['start_station_name', 'start_station_id']].mode().iloc[0]

mask_both_start_missing = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna()

df_rent.loc[mask_both_start_missing, 'start_station_id'] = most_common_start['start_station_id']
df_rent.loc[mask_both_start_missing, 'start_station_name'] = most_common_start['start_station_name']


In [30]:
# بناء قاموس من الاسم إلى المعرف (الأكثر شيوعًا)
end_name_to_id = (
    df_rent.dropna(subset=['end_station_id', 'end_station_name'])
    .groupby('end_station_name')['end_station_id']
    .agg(lambda x: x.mode().iloc[0])
)

# قناع للصفوف التي فيها الاسم موجود والمعرف مفقود
mask_eonly_id_missing = df_rent['end_station_id'].isna() & df_rent['end_station_name'].notna()

# تعبئة end_station_id
df_rent.loc[mask_eonly_id_missing, 'end_station_id'] = df_rent.loc[mask_eonly_id_missing, 'end_station_name'].map(end_name_to_id)


In [34]:
# الأكثر تكرارًا لكل start_station_id
most_common_end_per_start = (
    df_rent.dropna(subset=['start_station_id', 'end_station_id'])
    .groupby('start_station_id')['end_station_id']
    .agg(lambda x: x.mode().iloc[0])
)

# قناع للصفوف التي نهاية المحطة مفقودة كليًا
mask_missing_end = df_rent['end_station_id'].isna() & df_rent['end_station_name'].isna() & df_rent['start_station_id'].notna()

# تعبئة end_station_id بالتخمين من البداية
df_rent.loc[mask_missing_end, 'end_station_id'] = df_rent.loc[mask_missing_end, 'start_station_id'].map(most_common_end_per_start)
# بناء قاموس end_id إلى name
end_id_to_name = (
    df_rent.dropna(subset=['end_station_id', 'end_station_name'])
    .groupby('end_station_id')['end_station_name']
    .agg(lambda x: x.mode().iloc[0])
)

# تعبئة الاسم بعد تعبئة المعرف
df_rent.loc[mask_missing_end, 'end_station_name'] = df_rent.loc[mask_missing_end, 'end_station_id'].map(end_id_to_name)


In [35]:
print((df_rent['start_station_id'] == df_rent['end_station_id']).mean())

0.04967778117054006


##Exploration and Analysis

##Pattern discovery

##Documentation